In [ ]:
using System.IO;
using System.Text.Json;

In [ ]:
public class JsonColumn {
    public string Name { get; set; }
    public string Path { get; set; }
    public string DataType { get; set; }
    public JsonValueKind ValueKind { get; set; }

}

In [ ]:
var jsonString = File.ReadAllText("transactions.json");
// var jsonObject = JsonSerializer.Deserialize<Dictionary<string, object>>(jsonString);
jsonString

{
    "results": [
      {
        "id": "062b3a9b-b04f-4d9c-9ebc-81dcb5bbf285",
        "transactionId": "356deb9f-98e6-4d6f-8db3-b378241895a1",
        "note": "Direct income 38e8c917-0afc-4583-8ffe-1b53ad601197",
        "bankAccountRef": {
          "id": "ff8fdc4c-2aad-4ec1-9430-6bd201832eb3",
          "name": "Undeposited Funds"
        },
        "date": "2022-04-19T00:00:00",
        "status": "Unreconciled",
        "currency": "GBP",
        "currencyRate": 1,
        "lines": [
          {
            "description": "Payment for direct income $38e8c917-0afc-4583-8ffe-1b53ad601197",
            "recordRef": {
              "id": "38e8c917-0afc-4583-8ffe-1b53ad601197",
              "dataType": "directIncomes"
            },
            "amount": 188.33
          }
        ],
        "totalAmount": 188.33,
        "modifiedDate": "2023-04-26T12:38:18",
        "sourceModifiedDate": "2022-05-13T01:06:00",
        "metadata": {
          "isDeleted": false
        }
      },
  

In [ ]:
JsonDocument document = JsonDocument.Parse(jsonString);

In [ ]:
JsonElement rootElement = document.RootElement;
rootElement.ValueKind

Object

In [ ]:
rootElement.GetType()

System.Text.Json.JsonElement

In [ ]:
var rootChildren = rootElement.EnumerateObject();
foreach (var child in rootChildren)
{
    Console.WriteLine(child.Name+" : "+child.Value.ValueKind);
}

results : Array
pageNumber : Number
pageSize : Number
totalResults : Number
_links : Object


In [ ]:
var columns = new List<JsonColumn>();
var rootColumn = new JsonColumn
{
    Name = "",
    Path = "$",
    ValueKind = rootElement.ValueKind,
    DataType = $"root{rootElement.ValueKind}"
};
columns.Add(rootColumn);
var rootChildren = rootElement.EnumerateObject();
foreach (var child in rootChildren)
{
    ProcessJsonProperty(child, rootColumn.Path, columns);
};
columns

index value 0 Submission#22+JsonColumn Name Path $ DataType rootObject ValueKind Object 1 Submission#22+JsonColumn Name results Path $.['results'] DataType ObjectArray ValueKind Array 2 Submission#22+JsonColumn Name id Path $.['results'][].['id'] DataType string ValueKind String 3 Submission#22+JsonColumn Name transactionId Path $.['results'][].['transactionId'] DataType string ValueKind String 4 Submission#22+JsonColumn Name note Path $.['results'][].['note'] DataType string ValueKind String 5 Submission#22+JsonColumn Name bankAccountRef Path $.['results'][].['bankAccountRef'] DataType bankAccountRefObject ValueKind Object 6 Submission#22+JsonColumn Name id Path $.['results'][].['bankAccountRef'].['id'] DataType string ValueKind String 7 Submission#22+JsonColumn Name name Path $.['results'][].['bankAccountRef'].['name'] DataType string ValueKind String 8 Submission#22+JsonColumn Name date Path $.['results'][].['date'] DataType string ValueKind String 9 Submission#22+JsonColumn Name status Path $.['results'][].['status'] DataType string ValueKind String 10 Submission#22+JsonColumn Name currency Path $.['results'][].['currency'] DataType string ValueKind String 11 Submission#22+JsonColumn Name currencyRate Path $.['results'][].['currencyRate'] DataType double ValueKind Number 12 Submission#22+JsonColumn Name lines Path $.['results'][].['lines'] DataType ObjectArray ValueKind Array 13 Submission#22+JsonColumn Name description Path $.['results'][].['lines'][].['description'] DataType string ValueKind String 14 Submission#22+JsonColumn Name recordRef Path $.['results'][].['lines'][].['recordRef'] DataType recordRefObject ValueKind Object 15 Submission#22+JsonColumn Name id Path $.['results'][].['lines'][].['recordRef'].['id'] DataType string ValueKind String 16 Submission#22+JsonColumn Name dataType Path $.['results'][].['lines'][].['recordRef'].['dataType'] DataType string ValueKind String 17 Submission#22+JsonColumn Name amount Path $.['results'][].['lines'][].['amount'] DataType double ValueKind Number 18 Submission#22+JsonColumn Name totalAmount Path $.['results'][].['totalAmount'] DataType double ValueKind Number 19 Submission#22+JsonColumn Name modifiedDate Path $.['results'][].['modifiedDate'] DataType string ValueKind String (13 more)

In [ ]:
public IList<JsonColumn> ProcessJsonDocument(JsonDocument document)
{
    var columns = new List<JsonColumn>();
    var rootColumn = new JsonColumn
    {
        Name = "$",
        Path = "$",
        ValueKind = document.RootElement.ValueKind,
        DataType = $"root{document.RootElement.ValueKind}"
    };
    columns.Add(rootColumn);
    switch (document.RootElement.ValueKind)
    {
        case JsonValueKind.Object:
            var rootChildren = document.RootElement.EnumerateObject();
            foreach (var child in rootChildren)
            {
                ProcessJsonProperty(child, rootColumn.Path, columns);
            };
            break;
        case JsonValueKind.Array:
            ProcessJsonArray(document.RootElement, rootColumn, columns);
            break;
        default:
            throw new ArgumentException("Unsupported JSON value kind", nameof(document.RootElement));
    }
    
    return columns;
}

public void ProcessJsonProperty(JsonProperty jsonProperty, string parent, List<JsonColumn> columns)
{
    var column = new JsonColumn
    {
        Name = jsonProperty.Name,
        Path = $"{parent}.['{jsonProperty.Name}']",
        ValueKind = jsonProperty.Value.ValueKind
    };

    columns.Add(column);

    switch (jsonProperty.Value.ValueKind)
    {
        case JsonValueKind.String:
            column.DataType = "string";
            break;
        case JsonValueKind.Number:
            column.DataType = "double";
            break;
        case JsonValueKind.True:
        case JsonValueKind.False:
            column.DataType = "boolean";
            break;
        case JsonValueKind.Null:
        break;
        case JsonValueKind.Object:
            column.DataType = $"{jsonProperty.Name}Object";
            foreach (var child in jsonProperty.Value.EnumerateObject())
            {
                ProcessJsonProperty(child, column.Path, columns);
            }
            break;
        case JsonValueKind.Array:
            ProcessJsonArray(jsonProperty.Value, column, columns);
            break;
        default:
            throw new NotImplementedException("Unsupported JSON value kind: " + jsonProperty.Name);
    }
}

public void ProcessJsonArray(JsonElement jsonArray, JsonColumn parent, List<JsonColumn> columns) 
{
    var elements = jsonArray.EnumerateArray();
    if (!IsSameJsonValueKind(elements))
    {
        throw new ArgumentException("All elements in the array must have the same JSON value kind");
    } 
    else 
    {
        var first = elements.First();
        parent.DataType = $"{first.ValueKind}Array";
        switch (first.ValueKind) 
        {
            case JsonValueKind.String:
            case JsonValueKind.Number:
                parent.DataType = $"{first.ValueKind}Array";
                break;
            case JsonValueKind.True:
            case JsonValueKind.False:
                parent.DataType = "BooleanArray";
                break;
            case JsonValueKind.Null:
                parent.DataType = "NullArray";
                break;
            case JsonValueKind.Array:
                parent.DataType = "ArrayArray";
                break;
            case JsonValueKind.Object:
                parent.DataType = "ObjectArray";
                foreach (var prop in first.EnumerateObject())
                {
                    ProcessJsonProperty(prop, $"{parent.Path}[]", columns);
                }
                break;
            default:
                throw new ArgumentException("Unsupported JSON value kind");
        }
        
    }
}

public bool IsSameJsonValueKind(IEnumerable<JsonElement> elements)
{
    if (elements.Count() <= 1) return true;
    else return elements.All(x => x.ValueKind == elements.First().ValueKind);
}


In [ ]:
var rs = rootElement.GetProperty("results");
var childArr = rs.EnumerateArray();
var obj = childArr.All(x => x.ValueKind == JsonValueKind.Object);
// foreach (var item in childArr)
// {
//     var props = item.EnumerateObject();
//     foreach (var prop in props)
//     {
//         Console.WriteLine(prop.Name + ": " + prop.Value.ValueKind);
//     }
// }
obj

True

In [ ]:
var rsc = ProcessJsonDocument(document);
foreach (var row in rsc)
{
    Console.WriteLine(row.Path);
}


$
$.['results']
$.['results'][].['id']
$.['results'][].['transactionId']
$.['results'][].['note']
$.['results'][].['bankAccountRef']
$.['results'][].['bankAccountRef'].['id']
$.['results'][].['bankAccountRef'].['name']
$.['results'][].['date']
$.['results'][].['status']
$.['results'][].['currency']
$.['results'][].['currencyRate']
$.['results'][].['lines']
$.['results'][].['lines'][].['description']
$.['results'][].['lines'][].['recordRef']
$.['results'][].['lines'][].['recordRef'].['id']
$.['results'][].['lines'][].['recordRef'].['dataType']
$.['results'][].['lines'][].['amount']
$.['results'][].['totalAmount']
$.['results'][].['modifiedDate']
$.['results'][].['sourceModifiedDate']
$.['results'][].['metadata']
$.['results'][].['metadata'].['isDeleted']
$.['pageNumber']
$.['pageSize']
$.['totalResults']
$.['_links']
$.['_links'].['current']
$.['_links'].['current'].['href']
$.['_links'].['self']
$.['_links'].['self'].['href']
$.['_links'].['next']
$.['_links'].['next'].['href']
